In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.0
    Uninstalling openai-1.76.0:
      Successfully uninstalled openai-1.76.0


In [ ]:
# Install OpenAI library if not installed
# Import libraries
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import openai


In [ ]:
# Load the fine-tuned Silma v3 model
MODEL_PATH = "/content/drive/MyDrive/Silma_final_v3/checkpoint-6500"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE  = torch.float16 if DEVICE == "cuda" else torch.float32

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model     = AutoModelForCausalLM.from_pretrained(
                MODEL_PATH,
                torch_dtype=DTYPE,
                device_map="auto" if DEVICE == "cuda" else None
            ).to(DEVICE)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

...

model-00005-of-00005.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/d8/f7/d8f7b23da49e8d58e0d77665df7f184b5da73201b8120be5ca2c6bdc0ec5a41a/19272e9ca490cf69f9bdd24f2462a0a4780293924fb227cf9e0f84c95b01cf75?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00001-of-00005.safetensors%3B+filename%3D%22model-00001-of-00005.safetensors%22%3B&Expires=1745883041&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NTg4MzA0MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2Q4L2Y3L2Q4ZjdiMjNkYTQ5ZThkNThlMGQ3NzY2NWRmN2YxODRiNWRhNzMyMDFiODEyMGJlNWNhMmM2YmRjMGVjNWE0MWEvMTkyNzJlOWNhNDkwY2Y2OWY5YmRkMjRmMjQ2MmEwYTQ3ODAyOTM5MjRmYjIyN2NmOWUwZjg0Yzk1YjAxY2Y3NT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=LqLN-2-w1A4s7ozNnjPwbdMx%7EYGqPXdxg-orgzhL-La9d4HYIcV5pWHMXubctv2dQRjhjb3vM40vvuRFkRgf26HypksoPTvCJKTjinpwzro6nHPgMKKjL3ZuOYDZRebcQT2LHMzK8q15WkLLRNQpz14E0N49qzh4VM78OsrvXpXkj4fd2UJmurqiyxW025YEjy7E6fqd4YDkKEFiyGcL%7Ecocsu9QOrfLFM

model-00001-of-00005.safetensors:  25%|##4       | 975M/3.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
# Helper to build the input prompt
ST_BAYT_TOKEN = "<|bayt|>"
VERSE_TOKEN   = "<|verse|>"
ED_BAYT_TOKEN = "<|endbayt|>"

def build_prompt(title: str, meter: str, theme: str = "عامة", qafiyah: str = "م") -> str:
    return f"""أنشئ قصيدة عربية فصيحة وفقاً للمواصفات التالية:

العنوان: {title}
البحر: {meter}
نوع القصيدة: {theme}
القافية: {qafiyah}

يجب أن تكون القصيدة:
- ملتزمة بقواعد بحر {meter} وتفعيلاته
- منتهية كل بيت بحرف :{qafiyah}
- متناسبة مع موضوع :{title}

الهيكل المطلوب:
{ST_BAYT_TOKEN} الشطر الأول {VERSE_TOKEN} الشطر الثاني {ED_BAYT_TOKEN}

اكتب القصيدة:"""


In [ ]:
title = "جامعة سعود في نجد فخرها تجد"
meter = "الطويل"
theme = "قصيدة عامه"
qafiyah = "جد"

# Build the full prompt
prompt = build_prompt(title, meter, theme, qafiyah)

# Tokenize and generate
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
gen_ids = model.generate(
    **inputs,
    max_new_tokens=128,
    temperature=0.8,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)

# Decode the generated text
generated_text = tokenizer.decode(
    gen_ids[0][inputs.input_ids.shape[-1]:],
    skip_special_tokens=False
)

print("\nGenerated by Fine-tuned Model:")
print("="*80)
print(generated_text)
print("="*80)



Generated by Fine-tuned Model:


<|bayt|> جامِعَةٌ سعودٌ في نَجِدٍ فَخْرُها <|verse|> تَجْدُ في حُجَجِ الأُجْرِينِ نِعْمَ جَجِدِ <|endbayt|> <|bayt|> وَلَوْ أَنَّ مَخْطِوطَةً بِمِجْدِ جَوْهَرِها <|verse|> تَسْتَطِيعُ تَوَال


In [ ]:
import openai

# Set your OpenAI API Key
openai.api_key = ""  # ⬅️ Replace with your real key!

# Build the enhancement prompt for GPT
gpt_prompt = f"""
أنت خبير شعر عربي فصيح.

هذه قصيدة تم توليدها بواسطة نموذج لغوي:

\"\"\"
{generated_text}
\"\"\"

مهمتك:
- تحسين جودة الأبيات مع الحفاظ على البحر والقافية.
- إصلاح أي أخطاء لغوية أو عروضية.
- لا تغيّر الفكرة الأساسية للقصيدة.
- إذا وُجد تكرار في الأبيات أو الكلمات، فاحذفه بدون الإخلال بالمعنى.
- اكتب القصيدة المحسّنة فقط بدون أي شرح إضافي.
"""

# Request GPT to enhance
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "أنت محسن شعري محترف."},
        {"role": "user", "content": gpt_prompt}
    ],
    temperature=0.5,
    max_tokens=800
)

# Get enhanced poem
enhanced_poem = response['choices'][0]['message']['content']

print("\nEnhanced by GPT:")
print("="*80)
print(enhanced_poem)
print("="*80)



Enhanced by GPT:
جامعةٌ في نجدٍ، فخرها سعودٌ
تجد في حجج العلم نعم الجود
ومخطوطة العزّ بجوهرها معود
تستطيع أن تجود بالمجد الموعود
